In [1]:
#PP2 PCB Detection and Merging

#Import the libraries
import pandas as pd
import numpy as np
import scipy as sp

import os
import sys
import time

import csv
import datetime

import importlib

from matplotlib import pyplot as plt
from matplotlib import dates  as md 
from matplotlib.colors import LogNorm

In [2]:
#Import routines
import routine_rawaccl_to_binryseqPP2  
import routine_get_pcb_durPP2          
import routine_elim_spur_pcbPP2        
import routine_get_pcb_level_dataPP2   
import routine_merging_algoPP2         
import routine_merge_pxmty_algoPP2

Start of definition of the rawaccl_to_binryseq procedure 2020-05-08 18:40:24.845280
Version currently used was compiled at 2019-06-25 17:08
Inputs are inp_vibr_df,downsample_rate,use_filt_accl_flag,use_quant_sig_flag,HYS_LOW_THRESH,HYS_HIGH_THRESH
End   of definition of the procedure 2020-05-08 18:40:24.845546
Start of defining procedure : 2020-05-08 18:40:24.887264
End   of defining procedure : 2020-05-08 18:40:24.887383
Start of defining procedure : 2020-05-08 18:40:24.904972
End   of defining procedure : 2020-05-08 18:40:24.905153
Start of defining procedure : 2020-05-08 18:40:24.915963
Version : 2019-04-23 12:50
End   of defining procedure : 2020-05-08 18:40:24.916107
Start of defining procedure : 2020-05-08 18:40:24.926944
Using version 2019-04-06 08:30
End   of defining procedure : 2020-05-08 18:40:24.927102
Start of defining procedure : 2020-05-08 18:40:24.953655
Version : 2019-04-09 21:59
End   of defining procedure : 2020-05-08 18:40:24.953804


In [3]:
pp2_vibr_file = pd.read_csv('PP2_IMU0506.csv',usecols = ['time','ax','ay','az'])
pp2_vibr_file = pp2_vibr_file.sort_values(by="time")
pp2_vibr_file = pp2_vibr_file.reset_index()
pp2_vibr_file = pp2_vibr_file.assign(net_accl = np.sqrt(pp2_vibr_file['ax']**2 + pp2_vibr_file['ay']**2 + pp2_vibr_file['az']**2))

In [4]:
pp2_vibr_file['time']=pd.to_datetime(pp2_vibr_file['time'])
pp2_vibr_file=pp2_vibr_file.rename(columns={'time':'timestamp'})

In [5]:
pp2_vibr_file

,index,timestamp,ax,ay,az,net_accl
0,0,2020-05-06 02:00:00.010436096,-0.012756,1.009949,0.034363,1.010614
1,1,2020-05-06 02:00:00.016482048,-0.012756,1.009949,0.034363,1.010614
2,2,2020-05-06 02:00:00.017519104,-0.014526,1.011475,0.035645,1.012207
3,3,2020-05-06 02:00:00.018502912,-0.014526,1.011475,0.035645,1.012207
4,4,2020-05-06 02:00:00.019480832,-0.014526,1.011475,0.035645,1.012207
5,5,2020-05-06 02:00:00.020450048,-0.015564,1.009766,0.034363,1.010470
6,6,2020-05-06 02:00:00.021425920,-0.015564,1.009766,0.034363,1.010470
7,7,2020-05-06 02:00:00.066471168,-0.016357,1.011414,0.031860,1.012047
8,8,2020-05-06 02:00:00.067514880,-0.014954,1.010681,0.033264,1.011339
9,9,2020-05-06 02:00:00.068506112,-0.014954,1.010681,0.033264,1.011339


In [6]:
filt_sig_pp2 = routine_rawaccl_to_binryseqPP2.rawaccl_to_binryseq(pp2_vibr_file,1,'Y','Y',0.8,1.2)

START :  2020-05-08 18:40:49.032122
Mean      of normalized signal = 1.1721278016337648e-11
Deviation of normalized signal = 1.0000000000000702
-------------------------------------------------------
Performing Wiener filtering


/home/raghu/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


Performing downsampling by : 1
Performing quantization
Mode = 0.125
0 : 0.05 | 0
0.05 : 0.1 | 0
0.1 : 0.15000000000000002 | 1679987
0.15000000000000002 : 0.2 | 0
0.2 : 0.25 | 1456184
0.25 : 0.3 | 29642
0.3 : 0.35 | 17732
0.35 : 0.39999999999999997 | 11758
0.39999999999999997 : 0.44999999999999996 | 7960
0.44999999999999996 : 0.49999999999999994 | 7397
0.49999999999999994 : 0.5499999999999999 | 6051
0.5499999999999999 : 0.6 | 6526
0.6 : 0.65 | 8056
0.65 : 0.7000000000000001 | 15468
0.7000000000000001 : 0.7500000000000001 | 9547
0.7500000000000001 : 0.8000000000000002 | 11506
0.8000000000000002 : 0.8500000000000002 | 12919
0.8500000000000002 : 0.9000000000000002 | 12346
0.9000000000000002 : 0.9500000000000003 | 13023
0.9500000000000003 : 1.0000000000000002 | 13246
1.0000000000000002 : 1.0500000000000003 | 9955
1.0500000000000003 : 1.1000000000000003 | 7436
1.1000000000000003 : 1.1500000000000004 | 7629
1.1500000000000004 : 1.2000000000000004 | 7785
1.2000000000000004 : 1.2500000000000004

In [7]:
pcb_data_pp2 = routine_get_pcb_durPP2.get_pcb_dur(filt_sig_pp2.binry_sig)

End of processing at i = 115 j = 115


In [8]:
pcb_data_pp2

,arvl_index,dptr_index,pcb_actv_dur
0,499492,499847,355
1,501885,502182,297
2,507323,508427,1104
3,528951,529660,709
4,529925,530226,301
5,1448398,1448671,273
6,2233751,2233851,100
7,2234408,2236952,2544
8,2238449,2239118,669
9,2240191,2240781,590


In [9]:
filt_sig_pp2['cor_binry_sig'] = routine_elim_spur_pcbPP2.elim_spur_pcb(filt_sig_pp2.binry_sig,pcb_data_pp2,700)

/home/raghu/Desktop/TCSstreaming/iiotstream2/streaming2/PP2/routine_elim_spur_pcbPP2.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cor_binry_sig[STR_IND:END_IND] = 0.0


In [10]:
filt_sig_pp2

,timestamp,rlng_accl,filt_accl,quant_sig,trnry_sig,binry_sig,cor_binry_sig
0,2020-05-06 02:00:01.449500160,0.201551,0.186287,0.15,0.0,0.0,0.0
1,2020-05-06 02:00:01.450455040,0.201596,0.187140,0.15,0.0,0.0,0.0
2,2020-05-06 02:00:01.472012032,0.201642,0.187925,0.15,0.0,0.0,0.0
3,2020-05-06 02:00:01.476561920,0.201713,0.188665,0.15,0.0,0.0,0.0
4,2020-05-06 02:00:01.477572864,0.201518,0.189112,0.15,0.0,0.0,0.0
5,2020-05-06 02:00:01.478543872,0.201322,0.189501,0.15,0.0,0.0,0.0
6,2020-05-06 02:00:01.479515904,0.201183,0.189895,0.15,0.0,0.0,0.0
7,2020-05-06 02:00:01.520730112,0.201425,0.190564,0.15,0.0,0.0,0.0
8,2020-05-06 02:00:01.522329088,0.201764,0.191272,0.15,0.0,0.0,0.0
9,2020-05-06 02:00:01.523334912,0.202232,0.192041,0.15,0.0,0.0,0.0


In [11]:
pcb_data_pp2 = routine_get_pcb_durPP2.get_pcb_dur(filt_sig_pp2.cor_binry_sig)

End of processing at i = 57 j = 57


In [12]:
pcb_level_pp2 = routine_get_pcb_level_dataPP2.get_pcb_level_data(filt_sig_pp2,pcb_data_pp2,50)

START :  2020-05-08 18:42:26.586834
str_index = 507323
local_str = 0
end_index = 2796086
local_end = 56
Mode = 725.0
local_mode = 725.0
------------------------
Mode calculated at : 2020-05-08 18:42:27.108396
Number of boards detected        = 57
Machine utilization factor       = 5.062626207498773 %
Average   board active duration  = 41.66175438596491 secs
Median    board active duration  = 31.973333333333333 secs
Deviation board active duration  = 31.563730920040094 secs
END   :  2020-05-08 18:42:27.474857
Execution time = 0.014800383333333333 mins


In [13]:
pcb_level_pp2.reset_index(inplace=True)
pcb_level_pp2

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,1,10.940196,0.0,1.522759
1,1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,1,6.986179,0.0,0.977931
2,2,2234408,2020-05-06 08:20:54.636967936,2236952,2020-05-06 08:21:20.007554048,1,25.370586,0.0,3.508966
3,3,2242634,2020-05-06 08:22:19.149548032,2246759,2020-05-06 08:23:02.327467008,1,43.177919,0.0,5.689655
4,4,2417459,2020-05-06 08:51:29.537015040,2418520,2020-05-06 08:51:40.034030080,1,10.497015,0.0,1.463448
5,5,2418852,2020-05-06 08:51:43.249620992,2419584,2020-05-06 08:51:50.403249920,1,7.153629,0.0,1.009655
6,6,2419862,2020-05-06 08:51:53.178056960,2420645,2020-05-06 08:52:00.837554944,1,7.659498,0.0,1.080000
7,7,2422757,2020-05-06 08:52:21.608366848,2428283,2020-05-06 08:53:16.017597952,1,54.409231,0.0,7.622069
8,8,2429051,2020-05-06 08:53:23.497212928,2434750,2020-05-06 08:54:19.627592960,1,56.130380,0.0,7.860690
9,9,2435447,2020-05-06 08:54:26.509524224,2440047,2020-05-06 08:55:11.648305920,1,45.138782,0.0,6.344828


In [14]:
pcb_level_pp2.drop('index', axis=1, inplace=True)

In [15]:
tempdf=pcb_level_pp2
tempdf['nmodes']=0
tempdf

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,1,10.940196,0.0,1.522759,0
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,1,6.986179,0.0,0.977931,0
2,2234408,2020-05-06 08:20:54.636967936,2236952,2020-05-06 08:21:20.007554048,1,25.370586,0.0,3.508966,0
3,2242634,2020-05-06 08:22:19.149548032,2246759,2020-05-06 08:23:02.327467008,1,43.177919,0.0,5.689655,0
4,2417459,2020-05-06 08:51:29.537015040,2418520,2020-05-06 08:51:40.034030080,1,10.497015,0.0,1.463448,0
5,2418852,2020-05-06 08:51:43.249620992,2419584,2020-05-06 08:51:50.403249920,1,7.153629,0.0,1.009655,0
6,2419862,2020-05-06 08:51:53.178056960,2420645,2020-05-06 08:52:00.837554944,1,7.659498,0.0,1.080000,0
7,2422757,2020-05-06 08:52:21.608366848,2428283,2020-05-06 08:53:16.017597952,1,54.409231,0.0,7.622069,0
8,2429051,2020-05-06 08:53:23.497212928,2434750,2020-05-06 08:54:19.627592960,1,56.130380,0.0,7.860690,0
9,2435447,2020-05-06 08:54:26.509524224,2440047,2020-05-06 08:55:11.648305920,1,45.138782,0.0,6.344828,0


In [16]:
mod_actv_dur = routine_get_pcb_level_dataPP2.get_mode(pcb_data_pp2.pcb_actv_dur,50)/100

Mode = 725.0


In [17]:
mod_actv_dur 

7.25

In [18]:
tseries = np.zeros(tempdf.shape[0])
tseries

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [19]:
for i, row in pcb_level_pp2.iterrows():
    nmodes=row.proc_dur/mod_actv_dur
    frac=nmodes%1
    nmodes=np.floor(nmodes)
    
    if (1-frac) <= .15:
        nmodes=np.floor(nmodes)+1
    
    if nmodes==1.0:
        continue
    tseries[i] = nmodes
tempdf['nmodes'] = tseries

In [20]:
tseries

array([ 0.,  0.,  3.,  6.,  0.,  0.,  0.,  7.,  7.,  6.,  0.,  0.,  4.,
        2.,  0.,  0.,  4.,  2.,  6.,  2.,  6.,  3., 17.,  9.,  5.,  2.,
        3., 14.,  0.,  4.,  3.,  6.,  6.,  3.,  5.,  0.,  0.,  3.,  3.,
        7.,  7.,  0.,  6.,  6.,  0.,  0.,  0.,  7.,  0.,  6.,  2.,  4.,
        2.,  2.,  4.,  6.,  6.])

In [21]:
temp_df2=tempdf[tempdf['nmodes']>1]

In [22]:
temp_df2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
2,2234408,2020-05-06 08:20:54.636967936,2236952,2020-05-06 08:21:20.007554048,1,25.370586,0.0,3.508966,3.0
3,2242634,2020-05-06 08:22:19.149548032,2246759,2020-05-06 08:23:02.327467008,1,43.177919,0.0,5.689655,6.0
7,2422757,2020-05-06 08:52:21.608366848,2428283,2020-05-06 08:53:16.017597952,1,54.409231,0.0,7.622069,7.0
8,2429051,2020-05-06 08:53:23.497212928,2434750,2020-05-06 08:54:19.627592960,1,56.130380,0.0,7.860690,7.0
9,2435447,2020-05-06 08:54:26.509524224,2440047,2020-05-06 08:55:11.648305920,1,45.138782,0.0,6.344828,6.0
12,2465671,2020-05-06 08:59:23.837497856,2469104,2020-05-06 08:59:57.656704000,1,33.819206,0.0,4.735172,4.0
13,2469997,2020-05-06 09:00:06.480133120,2472010,2020-05-06 09:00:26.390660096,1,19.910527,0.0,2.776552,2.0
16,2476197,2020-05-06 09:01:07.567270912,2479745,2020-05-06 09:01:42.613240832,1,35.045970,0.0,4.893793,4.0
17,2481155,2020-05-06 09:01:56.530860800,2482781,2020-05-06 09:02:12.560471040,1,16.029610,0.0,2.242759,2.0
18,2487043,2020-05-06 09:02:54.427300096,2491955,2020-05-06 09:03:42.889986048,1,48.462686,0.0,6.775172,6.0


In [23]:
split_times=[]
for i, row in temp_df2.iterrows():
    nmodes=int(row.nmodes)
    split_times.append(np.linspace(row.arvl_tmstmp.to_datetime64().astype('int'),row.dptr_tmstmp.to_datetime64().astype('int'),num=row.nmodes+1))

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.


In [24]:
split_times

[array([1.58875325e+18, 1.58875326e+18, 1.58875327e+18, 1.58875328e+18]),
 array([1.58875334e+18, 1.58875335e+18, 1.58875335e+18, 1.58875336e+18,
        1.58875337e+18, 1.58875338e+18, 1.58875338e+18]),
 array([1.58875514e+18, 1.58875515e+18, 1.58875516e+18, 1.58875516e+18,
        1.58875517e+18, 1.58875518e+18, 1.58875519e+18, 1.58875520e+18]),
 array([1.58875520e+18, 1.58875521e+18, 1.58875522e+18, 1.58875523e+18,
        1.58875524e+18, 1.58875524e+18, 1.58875525e+18, 1.58875526e+18]),
 array([1.58875527e+18, 1.58875527e+18, 1.58875528e+18, 1.58875529e+18,
        1.58875530e+18, 1.58875530e+18, 1.58875531e+18]),
 array([1.58875556e+18, 1.58875557e+18, 1.58875558e+18, 1.58875559e+18,
        1.58875560e+18]),
 array([1.58875561e+18, 1.58875562e+18, 1.58875563e+18]),
 array([1.58875567e+18, 1.58875568e+18, 1.58875569e+18, 1.58875569e+18,
        1.58875570e+18]),
 array([1.58875572e+18, 1.58875572e+18, 1.58875573e+18]),
 array([1.58875577e+18, 1.58875578e+18, 1.58875579e+18, 1.5887

In [25]:
temp_df2.reset_index(inplace=True)

In [26]:
temp_df2

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,2,2234408,2020-05-06 08:20:54.636967936,2236952,2020-05-06 08:21:20.007554048,1,25.370586,0.0,3.508966,3.0
1,3,2242634,2020-05-06 08:22:19.149548032,2246759,2020-05-06 08:23:02.327467008,1,43.177919,0.0,5.689655,6.0
2,7,2422757,2020-05-06 08:52:21.608366848,2428283,2020-05-06 08:53:16.017597952,1,54.409231,0.0,7.622069,7.0
3,8,2429051,2020-05-06 08:53:23.497212928,2434750,2020-05-06 08:54:19.627592960,1,56.130380,0.0,7.860690,7.0
4,9,2435447,2020-05-06 08:54:26.509524224,2440047,2020-05-06 08:55:11.648305920,1,45.138782,0.0,6.344828,6.0
5,12,2465671,2020-05-06 08:59:23.837497856,2469104,2020-05-06 08:59:57.656704000,1,33.819206,0.0,4.735172,4.0
6,13,2469997,2020-05-06 09:00:06.480133120,2472010,2020-05-06 09:00:26.390660096,1,19.910527,0.0,2.776552,2.0
7,16,2476197,2020-05-06 09:01:07.567270912,2479745,2020-05-06 09:01:42.613240832,1,35.045970,0.0,4.893793,4.0
8,17,2481155,2020-05-06 09:01:56.530860800,2482781,2020-05-06 09:02:12.560471040,1,16.029610,0.0,2.242759,2.0
9,18,2487043,2020-05-06 09:02:54.427300096,2491955,2020-05-06 09:03:42.889986048,1,48.462686,0.0,6.775172,6.0


In [27]:
PP_boardeventsSplit = pd.DataFrame()
for i, row in temp_df2.iterrows():
    nmodes=int(row.nmodes)
    wt=row.proc_dur/nmodes
#     en=row.energy/nmodes
#     if nmodes%2==0:
    for j in range(len(split_times[i])-1):
        ts1=pd.to_datetime(split_times[i][j])
        ts2=pd.to_datetime(split_times[i][j+1])
        PP_boardeventsSplit=PP_boardeventsSplit.append({"arvl_index":row.arvl_index, "arvl_tmstmp":ts1, "dptr_index":row.dptr_index, "dptr_tmstmp":ts2, "no_of_parts":1, "proc_dur": wt, "maint_dur":row.maint_dur, "weightage":wt/mod_actv_dur, "nmodes":1}, ignore_index=True)

In [28]:
PP_boardeventsSplit

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
0,2234408.0,2020-05-06 08:20:54.636967936,2236952.0,2020-05-06 08:21:03.093829888,0.0,1.0,1.0,8.456862,1.166464
1,2234408.0,2020-05-06 08:21:03.093829888,2236952.0,2020-05-06 08:21:11.550692096,0.0,1.0,1.0,8.456862,1.166464
2,2234408.0,2020-05-06 08:21:11.550692096,2236952.0,2020-05-06 08:21:20.007554048,0.0,1.0,1.0,8.456862,1.166464
3,2242634.0,2020-05-06 08:22:19.149548032,2246759.0,2020-05-06 08:22:26.345867776,0.0,1.0,1.0,7.196320,0.992596
4,2242634.0,2020-05-06 08:22:26.345867776,2246759.0,2020-05-06 08:22:33.542187776,0.0,1.0,1.0,7.196320,0.992596
5,2242634.0,2020-05-06 08:22:33.542187776,2246759.0,2020-05-06 08:22:40.738507520,0.0,1.0,1.0,7.196320,0.992596
6,2242634.0,2020-05-06 08:22:40.738507520,2246759.0,2020-05-06 08:22:47.934827264,0.0,1.0,1.0,7.196320,0.992596
7,2242634.0,2020-05-06 08:22:47.934827264,2246759.0,2020-05-06 08:22:55.131147264,0.0,1.0,1.0,7.196320,0.992596
8,2242634.0,2020-05-06 08:22:55.131147264,2246759.0,2020-05-06 08:23:02.327467008,0.0,1.0,1.0,7.196320,0.992596
9,2422757.0,2020-05-06 08:52:21.608366848,2428283.0,2020-05-06 08:52:29.381114112,0.0,1.0,1.0,7.772747,1.072103


In [29]:
pcb_level_pp2.drop(tempdf[tempdf['nmodes']>1].index, inplace=True)

In [30]:
pcb_level_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,1,10.940196,0.0,1.522759,0.0
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,1,6.986179,0.0,0.977931,0.0
4,2417459,2020-05-06 08:51:29.537015040,2418520,2020-05-06 08:51:40.034030080,1,10.497015,0.0,1.463448,0.0
5,2418852,2020-05-06 08:51:43.249620992,2419584,2020-05-06 08:51:50.403249920,1,7.153629,0.0,1.009655,0.0
6,2419862,2020-05-06 08:51:53.178056960,2420645,2020-05-06 08:52:00.837554944,1,7.659498,0.0,1.080000,0.0
10,2462398,2020-05-06 08:58:51.570283008,2463375,2020-05-06 08:59:01.209817856,1,9.639535,0.0,1.347586,0.0
11,2463454,2020-05-06 08:59:01.998516992,2464426,2020-05-06 08:59:11.538362880,1,9.539846,0.0,1.340690,0.0
14,2473581,2020-05-06 09:00:41.723532800,2474601,2020-05-06 09:00:51.820549120,1,10.097016,0.0,1.406897,0.0
15,2475302,2020-05-06 09:00:58.736047104,2476120,2020-05-06 09:01:06.781317888,1,8.045271,0.0,1.128276,0.0
28,2607312,2020-05-06 09:22:36.707302144,2608390,2020-05-06 09:22:47.190637056,1,10.483335,0.0,1.486897,0.0


In [31]:
pcb_level_pp2=pcb_level_pp2.append(PP_boardeventsSplit, ignore_index=True)

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [32]:
pcb_level_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,0.0,0.0,1,10.940196,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,0.0,0.0,1,6.986179,0.977931
2,2417459,2020-05-06 08:51:29.537015040,2418520,2020-05-06 08:51:40.034030080,0.0,0.0,1,10.497015,1.463448
3,2418852,2020-05-06 08:51:43.249620992,2419584,2020-05-06 08:51:50.403249920,0.0,0.0,1,7.153629,1.009655
4,2419862,2020-05-06 08:51:53.178056960,2420645,2020-05-06 08:52:00.837554944,0.0,0.0,1,7.659498,1.080000
5,2462398,2020-05-06 08:58:51.570283008,2463375,2020-05-06 08:59:01.209817856,0.0,0.0,1,9.639535,1.347586
6,2463454,2020-05-06 08:59:01.998516992,2464426,2020-05-06 08:59:11.538362880,0.0,0.0,1,9.539846,1.340690
7,2473581,2020-05-06 09:00:41.723532800,2474601,2020-05-06 09:00:51.820549120,0.0,0.0,1,10.097016,1.406897
8,2475302,2020-05-06 09:00:58.736047104,2476120,2020-05-06 09:01:06.781317888,0.0,0.0,1,8.045271,1.128276
9,2607312,2020-05-06 09:22:36.707302144,2608390,2020-05-06 09:22:47.190637056,0.0,0.0,1,10.483335,1.486897


In [33]:
pcb_level_pp2 = pcb_level_pp2.sort_values(by="arvl_tmstmp")

In [34]:
pcb_level_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,nmodes,no_of_parts,proc_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,0.0,0.0,1,10.940196,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,0.0,0.0,1,6.986179,0.977931
17,2.23441e+06,2020-05-06 08:20:54.636967936,2.23695e+06,2020-05-06 08:21:03.093829888,0.0,1.0,1,8.456862,1.166464
18,2.23441e+06,2020-05-06 08:21:03.093829888,2.23695e+06,2020-05-06 08:21:11.550692096,0.0,1.0,1,8.456862,1.166464
19,2.23441e+06,2020-05-06 08:21:11.550692096,2.23695e+06,2020-05-06 08:21:20.007554048,0.0,1.0,1,8.456862,1.166464
20,2.24263e+06,2020-05-06 08:22:19.149548032,2.24676e+06,2020-05-06 08:22:26.345867776,0.0,1.0,1,7.196320,0.992596
21,2.24263e+06,2020-05-06 08:22:26.345867776,2.24676e+06,2020-05-06 08:22:33.542187776,0.0,1.0,1,7.196320,0.992596
22,2.24263e+06,2020-05-06 08:22:33.542187776,2.24676e+06,2020-05-06 08:22:40.738507520,0.0,1.0,1,7.196320,0.992596
23,2.24263e+06,2020-05-06 08:22:40.738507520,2.24676e+06,2020-05-06 08:22:47.934827264,0.0,1.0,1,7.196320,0.992596
24,2.24263e+06,2020-05-06 08:22:47.934827264,2.24676e+06,2020-05-06 08:22:55.131147264,0.0,1.0,1,7.196320,0.992596


In [35]:
pcb_level_pp2.reset_index(inplace=True)

In [36]:
pcb_level_pp2.drop('index', axis=1, inplace=True)
pcb_level_pp2.drop('nmodes', axis=1, inplace=True)

In [37]:
pcb_level_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,no_of_parts,proc_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,0.0,1,10.940196,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,0.0,1,6.986179,0.977931
2,2.23441e+06,2020-05-06 08:20:54.636967936,2.23695e+06,2020-05-06 08:21:03.093829888,0.0,1,8.456862,1.166464
3,2.23441e+06,2020-05-06 08:21:03.093829888,2.23695e+06,2020-05-06 08:21:11.550692096,0.0,1,8.456862,1.166464
4,2.23441e+06,2020-05-06 08:21:11.550692096,2.23695e+06,2020-05-06 08:21:20.007554048,0.0,1,8.456862,1.166464
5,2.24263e+06,2020-05-06 08:22:19.149548032,2.24676e+06,2020-05-06 08:22:26.345867776,0.0,1,7.196320,0.992596
6,2.24263e+06,2020-05-06 08:22:26.345867776,2.24676e+06,2020-05-06 08:22:33.542187776,0.0,1,7.196320,0.992596
7,2.24263e+06,2020-05-06 08:22:33.542187776,2.24676e+06,2020-05-06 08:22:40.738507520,0.0,1,7.196320,0.992596
8,2.24263e+06,2020-05-06 08:22:40.738507520,2.24676e+06,2020-05-06 08:22:47.934827264,0.0,1,7.196320,0.992596
9,2.24263e+06,2020-05-06 08:22:47.934827264,2.24676e+06,2020-05-06 08:22:55.131147264,0.0,1,7.196320,0.992596


In [38]:
pcb_merge_pp2 = pcb_level_pp2
mrg_binry_pp2 = filt_sig_pp2.cor_binry_sig

THRESHOLD = 0.95
HIGH_END  = 1.50

iter_one = 0
iter_two = 0

In [39]:
pcb_merge_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,maint_dur,no_of_parts,proc_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,0.0,1,10.940196,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,0.0,1,6.986179,0.977931
2,2.23441e+06,2020-05-06 08:20:54.636967936,2.23695e+06,2020-05-06 08:21:03.093829888,0.0,1,8.456862,1.166464
3,2.23441e+06,2020-05-06 08:21:03.093829888,2.23695e+06,2020-05-06 08:21:11.550692096,0.0,1,8.456862,1.166464
4,2.23441e+06,2020-05-06 08:21:11.550692096,2.23695e+06,2020-05-06 08:21:20.007554048,0.0,1,8.456862,1.166464
5,2.24263e+06,2020-05-06 08:22:19.149548032,2.24676e+06,2020-05-06 08:22:26.345867776,0.0,1,7.196320,0.992596
6,2.24263e+06,2020-05-06 08:22:26.345867776,2.24676e+06,2020-05-06 08:22:33.542187776,0.0,1,7.196320,0.992596
7,2.24263e+06,2020-05-06 08:22:33.542187776,2.24676e+06,2020-05-06 08:22:40.738507520,0.0,1,7.196320,0.992596
8,2.24263e+06,2020-05-06 08:22:40.738507520,2.24676e+06,2020-05-06 08:22:47.934827264,0.0,1,7.196320,0.992596
9,2.24263e+06,2020-05-06 08:22:47.934827264,2.24676e+06,2020-05-06 08:22:55.131147264,0.0,1,7.196320,0.992596


In [40]:
pp2_pxstr_file = pd.read_csv('PP2_prox0506.csv',usecols=['time','PP2Entry'])
pp2_pxstr_file=pp2_pxstr_file.rename(columns={'time':'timestamp'})
pp2_pxstr_file

,timestamp,PP2Entry
0,2020-05-06T08:21:33.002660096Z,1
1,2020-05-06T08:21:33.046422016Z,1
2,2020-05-06T08:21:33.136542976Z,1
3,2020-05-06T08:52:20.048254976Z,1
4,2020-05-06T08:53:19.732531968Z,1
5,2020-05-06T08:53:19.796463872Z,1
6,2020-05-06T08:53:19.9457792Z,1
7,2020-05-06T08:54:22.061594112Z,1
8,2020-05-06T08:54:22.129753088Z,1
9,2020-05-06T08:59:20.20893184Z,1


In [41]:
# pcb_level_pp2 = pcb_level_df
# pcb_merge_pp2 = pcb_level_pp2
# mrg_binry_pp2 = filt_sig_df.cor_binry_sig

In [42]:
# THRESHOLD = 0.95
# HIGH_END  = 1.50
# inp_df = pcb_merge_pp2 
# pxmty_df= pp2_pxstr_file
# inp_binry_sig = mrg_binry_pp2

In [41]:
PROC_ONE_END_FLAG = 0
while (PROC_ONE_END_FLAG == 0):
    
    iter_one = iter_one + 1
    print('PROC-ONE-ITERATION :', iter_one)
    
    pcb_merge_pp2,mrg_binry_pp2,no_of_merged_pcbs = routine_merge_pxmty_algoPP2.merge_pxmty_algo    \
    (inp_df        = pcb_merge_pp2                                      \
    ,pxmty_df      = pp2_pxstr_file      \
    ,inp_binry_sig = mrg_binry_pp2                         \
    ,THRESHOLD     = THRESHOLD                                               \
    ,HIGH_END      = HIGH_END                                                \
    )
    
    if (no_of_merged_pcbs == 0):
        PROC_ONE_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-ONE-ITERATION : 1
START :  2020-05-08 18:42:32.300607
No. of merged PCBS = 0
END   :  2020-05-08 18:42:33.036600
Execution time = 0.01226655 mins


In [42]:
pcb_merge_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,1,10.940196,0.0,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,1,6.986179,0.0,0.977931
2,2.23441e+06,2020-05-06 08:20:54.636967936,2.23695e+06,2020-05-06 08:21:03.093829888,1,8.456862,0.0,1.166464
3,2.23441e+06,2020-05-06 08:21:03.093829888,2.23695e+06,2020-05-06 08:21:11.550692096,1,8.456862,0.0,1.166464
4,2.23441e+06,2020-05-06 08:21:11.550692096,2.23695e+06,2020-05-06 08:21:20.007554048,1,8.456862,0.0,1.166464
5,2.24263e+06,2020-05-06 08:22:19.149548032,2.24676e+06,2020-05-06 08:22:26.345867776,1,7.196320,0.0,0.992596
6,2.24263e+06,2020-05-06 08:22:26.345867776,2.24676e+06,2020-05-06 08:22:33.542187776,1,7.196320,0.0,0.992596
7,2.24263e+06,2020-05-06 08:22:33.542187776,2.24676e+06,2020-05-06 08:22:40.738507520,1,7.196320,0.0,0.992596
8,2.24263e+06,2020-05-06 08:22:40.738507520,2.24676e+06,2020-05-06 08:22:47.934827264,1,7.196320,0.0,0.992596
9,2.24263e+06,2020-05-06 08:22:47.934827264,2.24676e+06,2020-05-06 08:22:55.131147264,1,7.196320,0.0,0.992596


In [43]:
PROC_TWO_END_FLAG = 0
while(PROC_TWO_END_FLAG == 0):
    
    iter_two = iter_two + 1
    print('PROC-TWO-ITERATION :', iter_two)
    
    pcb_merge_pp2,mrg_binry_pp2,no_of_merged_pcbs = routine_merging_algoPP2.merging_algo  \
    (inp_df        = pcb_merge_pp2                                   \
    ,inp_binry_sig = mrg_binry_pp2                                   \
    ,THRESHOLD     = THRESHOLD                                            \
    ,HIGH_END      = HIGH_END                                             \
    ) 
    
    if(no_of_merged_pcbs == 0):
        PROC_TWO_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-TWO-ITERATION : 1
START :  2020-05-08 18:42:33.171622
No. of merged PCBS = 0
END   :  2020-05-08 18:42:33.967932
Execution time = 0.013271833333333333 mins


In [44]:
pcb_merge_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,507323,2020-05-06 03:24:34.439613184,508427,2020-05-06 03:24:45.379809024,1,10.940196,0.0,1.522759
1,528951,2020-05-06 03:28:09.134403840,529660,2020-05-06 03:28:16.120583168,1,6.986179,0.0,0.977931
2,2.23441e+06,2020-05-06 08:20:54.636967936,2.23695e+06,2020-05-06 08:21:03.093829888,1,8.456862,0.0,1.166464
3,2.23441e+06,2020-05-06 08:21:03.093829888,2.23695e+06,2020-05-06 08:21:11.550692096,1,8.456862,0.0,1.166464
4,2.23441e+06,2020-05-06 08:21:11.550692096,2.23695e+06,2020-05-06 08:21:20.007554048,1,8.456862,0.0,1.166464
5,2.24263e+06,2020-05-06 08:22:19.149548032,2.24676e+06,2020-05-06 08:22:26.345867776,1,7.196320,0.0,0.992596
6,2.24263e+06,2020-05-06 08:22:26.345867776,2.24676e+06,2020-05-06 08:22:33.542187776,1,7.196320,0.0,0.992596
7,2.24263e+06,2020-05-06 08:22:33.542187776,2.24676e+06,2020-05-06 08:22:40.738507520,1,7.196320,0.0,0.992596
8,2.24263e+06,2020-05-06 08:22:40.738507520,2.24676e+06,2020-05-06 08:22:47.934827264,1,7.196320,0.0,0.992596
9,2.24263e+06,2020-05-06 08:22:47.934827264,2.24676e+06,2020-05-06 08:22:55.131147264,1,7.196320,0.0,0.992596


In [45]:
filt_sig_pp2['mrg_binry_sig'] = mrg_binry_pp2

In [46]:
mrg_binry_pp2

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
          ... 
3517986    0.0
3517987    0.0
3517988    0.0
3517989    0.0
3517990    0.0
3517991    0.0
3517992    0.0
3517993    0.0
3517994    0.0
3517995    0.0
3517996    0.0
3517997    0.0
3517998    0.0
3517999    0.0
3518000    0.0
3518001    0.0
3518002    0.0
3518003    0.0
3518004    0.0
3518005    0.0
3518006    0.0
3518007    0.0
3518008    0.0
3518009    0.0
3518010    0.0
3518011    0.0
3518012    0.0
3518013    0.0
3518014    0.0
3518015    0.0
Name: cor_binry_sig, Length: 3518016, dtype: float64

In [49]:
# pcb_merge_pp2 = pcb_merge_pp2[pcb_merge_pp2['weightage'] >= 0.75]

In [50]:
pp2EMfile = pd.read_csv('PP2_EM11.csv', usecols=['time', 'c1', 'c2', 'c3'])

In [51]:
raw_pp_current=pp2EMfile

In [52]:
raw_pp_current['time']=pd.to_datetime(raw_pp_current['time'])
raw_pp_current=raw_pp_current.rename(columns={'time':'timestamp'})

In [53]:
raw_pp_current

,timestamp,c1,c2,c3
0,2020-03-11 02:00:01.056186880,0.0,0.0,0.0
1,2020-03-11 02:00:02.911838976,0.0,0.0,0.0
2,2020-03-11 02:00:04.751537152,0.0,0.0,0.0
3,2020-03-11 02:00:06.591301888,0.0,0.0,0.0
4,2020-03-11 02:00:08.399424000,0.0,0.0,0.0
5,2020-03-11 02:00:10.254696960,0.0,0.0,0.0
6,2020-03-11 02:00:12.094251008,0.0,0.0,0.0
7,2020-03-11 02:00:13.949920768,0.0,0.0,0.0
8,2020-03-11 02:00:15.805701888,0.0,0.0,0.0
9,2020-03-11 02:00:17.661243904,0.0,0.0,0.0


In [54]:
raw_pp_current['total_current']=raw_pp_current['c1']+raw_pp_current['c2']+raw_pp_current['c3']

In [55]:
raw_pp_current

,timestamp,c1,c2,c3,total_current
0,2020-03-11 02:00:01.056186880,0.0,0.0,0.0,0.0
1,2020-03-11 02:00:02.911838976,0.0,0.0,0.0,0.0
2,2020-03-11 02:00:04.751537152,0.0,0.0,0.0,0.0
3,2020-03-11 02:00:06.591301888,0.0,0.0,0.0,0.0
4,2020-03-11 02:00:08.399424000,0.0,0.0,0.0,0.0
5,2020-03-11 02:00:10.254696960,0.0,0.0,0.0,0.0
6,2020-03-11 02:00:12.094251008,0.0,0.0,0.0,0.0
7,2020-03-11 02:00:13.949920768,0.0,0.0,0.0,0.0
8,2020-03-11 02:00:15.805701888,0.0,0.0,0.0,0.0
9,2020-03-11 02:00:17.661243904,0.0,0.0,0.0,0.0


In [56]:
working_times_df=pd.DataFrame()

working_times_df['timestamp']=pcb_merge_pp2['arvl_tmstmp']
working_times_df['working_time']=pcb_merge_pp2['proc_dur']
working_times_df['event']=1

In [57]:
working_times_df

,timestamp,working_time,event
0,2020-03-11 03:50:49.759336960,41.771533,1
1,2020-03-11 04:23:15.609496064,45.401074,1
2,2020-03-11 04:28:59.350942976,46.341115,1
3,2020-03-11 04:34:14.087979008,38.894107,1
4,2020-03-11 04:37:15.850456064,39.667192,1
5,2020-03-11 04:51:22.251493888,44.286317,1
6,2020-03-11 04:52:52.590245888,41.507379,1
7,2020-03-11 04:54:11.707623936,45.882484,1
8,2020-03-11 04:55:07.980244992,54.008706,1
9,2020-03-11 04:59:00.743568896,69.828800,1


In [58]:
working_times_df['energy'] = pcb_merge_pp2.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.dptr_tmstmp) & (x.arvl_tmstmp <= raw_pp_current.timestamp), ['total_current']].sum()*230/3600000, axis=1)

In [59]:
working_times_df

,timestamp,working_time,event,energy
0,2020-03-11 03:50:49.759336960,41.771533,1,0.003691
1,2020-03-11 04:23:15.609496064,45.401074,1,0.004426
2,2020-03-11 04:28:59.350942976,46.341115,1,0.004689
3,2020-03-11 04:34:14.087979008,38.894107,1,0.004197
4,2020-03-11 04:37:15.850456064,39.667192,1,0.009671
5,2020-03-11 04:51:22.251493888,44.286317,1,0.004289
6,2020-03-11 04:52:52.590245888,41.507379,1,0.004110
7,2020-03-11 04:54:11.707623936,45.882484,1,0.004889
8,2020-03-11 04:55:07.980244992,54.008706,1,0.004461
9,2020-03-11 04:59:00.743568896,69.828800,1,0.005250


In [60]:
PP_temp_idle=filt_sig_pp2[['timestamp', 'mrg_binry_sig']]
PP_temp_idle['mrg_binry_sig']=PP_temp_idle['mrg_binry_sig']*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [61]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.460818944,-0.0
1,2020-03-11 02:00:01.461822976,-0.0
2,2020-03-11 02:00:01.485358848,-0.0
3,2020-03-11 02:00:01.487150848,-0.0
4,2020-03-11 02:00:01.488193024,-0.0
5,2020-03-11 02:00:01.489198080,-0.0
6,2020-03-11 02:00:01.490236928,-0.0
7,2020-03-11 02:00:01.534104064,-0.0
8,2020-03-11 02:00:01.535755008,-0.0
9,2020-03-11 02:00:01.556931072,-0.0


In [62]:
PP_temp_idle['mrg_binry_sig'].iloc[0]=(PP_temp_idle['mrg_binry_sig'].iloc[1]+1)%2*-1
PP_temp_idle['mrg_binry_sig'].iloc[-1]=(PP_temp_idle['mrg_binry_sig'].iloc[-2]+1)%2*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.460818944,-1.0
1,2020-03-11 02:00:01.461822976,-0.0
2,2020-03-11 02:00:01.485358848,-0.0
3,2020-03-11 02:00:01.487150848,-0.0
4,2020-03-11 02:00:01.488193024,-0.0
5,2020-03-11 02:00:01.489198080,-0.0
6,2020-03-11 02:00:01.490236928,-0.0
7,2020-03-11 02:00:01.534104064,-0.0
8,2020-03-11 02:00:01.535755008,-0.0
9,2020-03-11 02:00:01.556931072,-0.0


In [64]:
idle_times_raw=sp.signal.find_peaks(PP_temp_idle.mrg_binry_sig, height=(-0.4), width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/100})

In [65]:
idle_times_raw_df

,sample_number,working_time
0,332884,6657.67
1,765339,1906.75
2,879687,290.52
3,912181,268.60
4,936723,143.59
5,994366,688.07
6,1035428,45.35
7,1043730,37.96
8,1050706,10.33
9,1063877,130.87


In [66]:
for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'start_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'timestamp']=PP_temp_idle.iloc[int(idle_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [67]:
idle_times_raw_df['event']=0

In [68]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [69]:
idle_times_raw_df

,sample_number,working_time,start_time,end_time,timestamp,event,energy
0,332884,6657.67,2020-03-11 02:00:01.460818944,2020-03-11 03:50:49.758272000,2020-03-11 02:55:06.972438016,0,0.152763
1,765339,1906.75,2020-03-11 03:51:31.529865984,2020-03-11 04:23:15.608467200,2020-03-11 04:07:14.607876096,0,0.092413
2,879687,290.52,2020-03-11 04:24:01.009574912,2020-03-11 04:28:59.349926912,2020-03-11 04:26:31.073171968,0,0.016176
3,912181,268.60,2020-03-11 04:29:45.691070976,2020-03-11 04:34:14.042170112,2020-03-11 04:32:01.069895936,0,0.014551
4,936723,143.59,2020-03-11 04:34:52.981080064,2020-03-11 04:37:15.849449984,2020-03-11 04:36:04.646307072,0,0.008774
5,994366,688.07,2020-03-11 04:39:55.790369024,2020-03-11 04:51:22.250484992,2020-03-11 04:45:39.339436032,0,0.037041
6,1035428,45.35,2020-03-11 04:52:06.492413952,2020-03-11 04:52:52.589235968,2020-03-11 04:52:29.627003904,0,0.002820
7,1043730,37.96,2020-03-11 04:53:34.042532864,2020-03-11 04:54:11.703510016,2020-03-11 04:53:52.989552128,0,0.002639
8,1050706,10.33,2020-03-11 04:54:57.589123072,2020-03-11 04:55:07.979238912,2020-03-11 04:55:02.712069120,0,0.000520
9,1063877,130.87,2020-03-11 04:56:09.929184000,2020-03-11 04:58:21.410420992,2020-03-11 04:57:15.311328768,0,0.006819


In [70]:
idle_times_raw_df['energy'].sum(axis = 0, skipna = True) 

1.3260062222222222

In [71]:
working_times_df=working_times_df[['event', 'working_time', 'energy','timestamp']]
idle_times_df=idle_times_raw_df[['timestamp','event', 'working_time', 'energy']]

In [72]:
working_times_df

,event,working_time,energy,timestamp
0,1,41.771533,0.003691,2020-03-11 03:50:49.759336960
1,1,45.401074,0.004426,2020-03-11 04:23:15.609496064
2,1,46.341115,0.004689,2020-03-11 04:28:59.350942976
3,1,38.894107,0.004197,2020-03-11 04:34:14.087979008
4,1,39.667192,0.009671,2020-03-11 04:37:15.850456064
5,1,44.286317,0.004289,2020-03-11 04:51:22.251493888
6,1,41.507379,0.004110,2020-03-11 04:52:52.590245888
7,1,45.882484,0.004889,2020-03-11 04:54:11.707623936
8,1,54.008706,0.004461,2020-03-11 04:55:07.980244992
9,1,69.828800,0.005250,2020-03-11 04:59:00.743568896


In [73]:
working_times_df['energy'].sum(axis = 0, skipna = True)

0.29796883333333335

In [74]:
# idle_times_df.reset_index(inplace=True)
# idle_times_df
# idle_times_df.drop('index', axis=1, inplace=True)
# idle_times_df

In [75]:
from influxdb import InfluxDBClient
import json

IFhost="localhost"
IFport=8086
IFdbname = 'IIOT'

client = InfluxDBClient(host=IFhost, port=IFport,database=IFdbname)

In [76]:
idle_events_batch=[]
processing_events_batch=[]
# events_batch = json.loads(LL_events.to_json())

for i,row in idle_times_df.iterrows():
    idle_events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP2'}, 'time':row.timestamp ,'fields': {'PP2 Idle Event':int(row.event),'PP2 Idle Time':float(row.working_time),"PP2 Idle Energy":float(row.energy)}})
    
for i,row in working_times_df.iterrows():
    processing_events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP2'}, 'time':row.timestamp ,'fields': {'PP2 Processing Event':int(row.event),'PP2 Processing Time':float(row.working_time),"PP2 Processing Energy":float(row.energy)}})

In [77]:
idle_events_batch

[{'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 02:55:06.972438016'),
  'fields': {'PP2 Idle Event': 0,
   'PP2 Idle Time': 6657.67,
   'PP2 Idle Energy': 0.15276280555555558}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:07:14.607876096'),
  'fields': {'PP2 Idle Event': 0,
   'PP2 Idle Time': 1906.75,
   'PP2 Idle Energy': 0.09241336111111112}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:26:31.073171968'),
  'fields': {'PP2 Idle Event': 0,
   'PP2 Idle Time': 290.52,
   'PP2 Idle Energy': 0.016176027777777775}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:32:01.069895936'),
  'fields': {'PP2 Idle Event': 0,
   'PP2 Idle Time': 268.6,
   'PP2 Idle Energy': 0.014550694444444444}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:36:04.646307072'),
  'fields': {'

In [78]:
processing_events_batch

[{'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 03:50:49.759336960'),
  'fields': {'PP2 Processing Event': 1,
   'PP2 Processing Time': 41.771533056,
   'PP2 Processing Energy': 0.0036915}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:23:15.609496064'),
  'fields': {'PP2 Processing Event': 1,
   'PP2 Processing Time': 45.40107392,
   'PP2 Processing Energy': 0.004426222222222222}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:28:59.350942976'),
  'fields': {'PP2 Processing Event': 1,
   'PP2 Processing Time': 46.341115136,
   'PP2 Processing Energy': 0.0046888055555555554}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP2'},
  'time': Timestamp('2020-03-11 04:34:14.087979008'),
  'fields': {'PP2 Processing Event': 1,
   'PP2 Processing Time': 38.89410688,
   'PP2 Processing Energy': 0.004196861111111112}},
 {'measurement': 'EVENTS',
  'tags': {'M

In [79]:
client.write_points(idle_events_batch,batch_size=len(idle_times_df))
client.write_points(processing_events_batch,batch_size=len(working_times_df))

True

In [80]:
PP_events=pd.DataFrame()

In [81]:
PP_events=PP_events.append(working_times_df)

In [82]:
PP_events=PP_events.append(idle_times_df)

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [83]:
PP_events

,energy,event,timestamp,working_time
0,0.003691,1,2020-03-11 03:50:49.759336960,41.771533
1,0.004426,1,2020-03-11 04:23:15.609496064,45.401074
2,0.004689,1,2020-03-11 04:28:59.350942976,46.341115
3,0.004197,1,2020-03-11 04:34:14.087979008,38.894107
4,0.009671,1,2020-03-11 04:37:15.850456064,39.667192
5,0.004289,1,2020-03-11 04:51:22.251493888,44.286317
6,0.004110,1,2020-03-11 04:52:52.590245888,41.507379
7,0.004889,1,2020-03-11 04:54:11.707623936,45.882484
8,0.004461,1,2020-03-11 04:55:07.980244992,54.008706
9,0.005250,1,2020-03-11 04:59:00.743568896,69.828800
